In [ ]:
!pip install datasets
!pip install yake

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
import yake

In [ ]:
import nltk
from transformers import AutoTokenizer, AutoModelForMaskedLM
from tqdm import tqdm
from nltk.corpus import stopwords
nltk.download('stopwords')
from collections import defaultdict
from typing import List
import string
from nltk.tokenize import word_tokenize
import os

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
signals_df = load_dataset("Salt-NLP/silent_signals",split = "train")
df = pd.DataFrame(signals_df)
df = df.drop(columns = ["party", "chamber", "speaker"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.37k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16258 [00:00<?, ? examples/s]

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df

,dog_whistle,dog_whistle_root,ingroup,content,date,subreddit,source,definition,type
0,SJW,social justice warrior,anti-liberal,Far left SJW/Woke activists use a variety of w...,06/03/2023,goldandblack,PRAW API,liberals,Informal
1,SJW,social justice warrior,anti-liberal,1) Relax. We don’t need SJW-lites commenting o...,08/03/2017,The_Donald,PRAW API,liberals,Informal
2,SJW,social justice warrior,anti-liberal,"Sure, Tifa is extreme (Advent Children is fine...",01/30/2016,TumblrInAction,PRAW API,liberals,Informal
3,SJWs,social justice warrior,anti-liberal,Sure theyve made a lot of decisions I as a gam...,06/15/2015,TumblrInAction,PRAW API,liberals,Informal
4,SJWs,social justice warrior,anti-liberal,I disagree. Wanting fair treatment and the fur...,08/31/2015,TumblrInAction,PRAW API,liberals,Informal
...,...,...,...,...,...,...,...,...,...
16253,law and order,law and order,racist,There were certainly people who meant that the...,09/08/2023,walkaway,PRAW API,Black and Latino people are lawless and commit...,Informal
16254,Baltimore,references to cities with large racial minorit...,racist,And how exactly do we improve it? And don't sa...,09/06/2023,walkaway,PRAW API,Minorities are dirty and criminals,Informal
16255,Baltimore,references to cities with large racial minorit...,racist,Yeah it probably is a bad idea in a country th...,02/04/2017,consoom,PRAW API,Minorities are dirty and criminals,Informal
16256,Baltimore,references to cities with large racial minorit...,racist,Photo Op probably took place in Whiteyville su...,09/06/2023,consoom,PRAW API,Minorities are dirty and criminals,Informal


In [ ]:
class Metrics:
  def __init__(self,file: str):
    self.metrics = MetricsOther(file)
  def measure_precision(
      self, predicted_dogwhistles: List[str], gold_dogwhistles: List[str]
  ) -> float:
    return self.metrics.measure_precision(predicted_dogwhistles, gold_dogwhistles)
  def measure_recall(
      self, predicted_dogwhistles: List[str], gold_dogwhistles: List[str]
  ) -> float:
    return self.metrics.measure_recall(predicted_dogwhistles, gold_dogwhistles)
  def measure_possible_recall(
      self, predicted_dogwhistles: List[str], gold_dogwhistles: List[str]
  ) -> float:
    return self.metrics.measure_possible_recall(predicted_dogwhistles, gold_dogwhistles,max_possible_ngrams)

class MetricsOther:
  def __init__(self,dogwhistle_file_path: str) -> None:
    self.dogwhistle_file_path = dogwhistle_file_path
    dogwhistles_df = pd.read_csv(self.dogwhistle_file_path, sep="\t")

    dogwhistle_set = dogwhistles_df["Surface Forms"].str.split(";").tolist()

    comparison_set = [
        x.strip().lower() for x in dogwhistles_df["Dogwhistle"].tolist()
    ]
    self.dogwhistle_to_surface = defaultdict(list)

    for i in range(len(dogwhistle_set)):
      self.dogwhistle_to_surface[comparison_set[i]] = set(
          [x.strip().lower() for x in dogwhistle_set[i]] + [comparison_set[i]]
      )

    self.ngrams = {
        x: min([len(word_tokenize(y)) for y in self.dogwhistle_to_surface[x]])
        for x in comparison_set
    }

    self.dogwhistles = dict()

    for i in range(len(dogwhistle_set)):
      for surface_form in dogwhistle_set[i]:
        self.dogwhistles[surface_form.strip().lower()] = comparison_set[i]
      self.dogwhistles[comparison_set[i]] = comparison_set[i]


  def measure_precision(
      self, predicted_dogwhistles: List[str], gold_dogwhistles: List[str]
  ) -> float:
    dogwhistle_map = {
        x.strip().lower(): self.dogwhistles[x.strip().lower()]
        for x in gold_dogwhistles
    }

    predicted_dogwhistles_cleaned = [
        x.strip().lower() for x in predicted_dogwhistles
    ]

    dogwhistles_found = []

    for dogwhistle in predicted_dogwhistles_cleaned:
      if dogwhistle in dogwhistle_map:
        dogwhistles_found.append(dogwhistle_map[dogwhistle])

    if len(predicted_dogwhistles) == 0:
      return 0

    return len(dogwhistles_found) / len(predicted_dogwhistles_cleaned)


  def measure_recall(
      self, predicted_dogwhistles: List[str], gold_dogwhistles: List[str]
  ) -> float:
    dogwhistle_map = {x: self.dogwhistles[x] for x in gold_dogwhistles}

    predicted_dogwhistles_cleaned = [
        x.strip().lower() for x in predicted_dogwhistles
    ]

    dogwhistles_found = []

    for dogwhistle in predicted_dogwhistles_cleaned:
      if dogwhistle in dogwhistle_map:
        dogwhistles_found.append(dogwhistle_map[dogwhistle])

    return len(set(dogwhistles_found)) / len(set(dogwhistle_map.values()))

  def measure_possible_recall(
      self, predicted_dogwhistles: List[str], gold_dogwhistles: List[str], max_possible_ngrams: int
  ) -> float:
    dogwhistle_map = {x: self.dogwhistles[x] for x in gold_dogwhistles}

    predicted_dogwhistles_cleaned = [
        x.strip().lower() for x in predicted_dogwhistles
    ]

    possible = [
        x
        for x in set(dogwhistle_map.values())
        if self.ngrams[x] <= max_possible_ngrams
    ]

    dogwhistles_found = []

    for dogwhistle in predicted_dogwhistles_cleaned:
      if dogwhistle in dogwhistle_map:
        dogwhistles_found.append(dogwhistle_map[dogwhistle])

    return len(set(dogwhistles_found)) / len(possible)


In [ ]:
class DWdetector:
  def __init__(self,given_keywords: List[str],
               data: List[str],thres: int,
               model_name: str,):
    self.given_keywords = given_keywords
    self.data = data
    self.thres = thres
    self.model_name = model_name
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.bert_tokenizer = AutoTokenizer.from_pretrained(self.model_name)
    self.bert_model = AutoModelForMaskedLM.from_pretrained(self.model_name).to(self.device)
    self.PAD = self.bert_tokenizer.pad_token
    self.MASK = self.bert_tokenizer.mask_token
    self.CLS = self.bert_tokenizer.cls_token
    self.SEP = self.bert_tokenizer.sep_token
  def single_MLM(self,message,threshold):
    max_length = self.bert_model.config.max_position_embeddings - 2
    tokens = self.bert_tokenizer(message,truncation=True, max_length=max_length, return_tensors = "pt")
    tokenized = self.bert_tokenizer.tokenize(message, truncation=True)[:max_length]
    tokens.to(self.device)
    with torch.no_grad():
      output = self.bert_model(**tokens)
    logits = output.logits
    logits = logits.squeeze(0)
    probs = torch.softmax(logits, dim=-1)

    out = []

    for idx, token in enumerate(tokenized):
      if token.strip() == self.MASK:
        tpok_prob, topk_indicies = torch.topk(probs[idx, :], threshold)
        topk_tokens = self.bert_tokenizer.convert_ids_to_tokens(topk_indicies.cpu().numpy())
        out = [[topk_tokens[i],float(tpok_prob[i])] for i in range(threshold)]
      return out
  def MLM(self, sgs, input_keywords, thres=1, filter_uninformative=0):
    MLM_score = defaultdict(float)
    sgs = [x.replace("[MASK]", self.MASK) for x in sgs]
    temp = sgs if len(sgs) < 10 else tqdm(sgs)
    skip_ms_num = 0
    good_sgs = []
    for sgs_i in temp:
      top_words = self.single_MLM(sgs_i, thres)
      seen_input = 0
      for input_i in input_keywords:
        if input_i in [x[0] for x in top_words[:thres]]:
          seen_input += 1
      if filter_uninformative == 1 and seen_input < 2:
        skip_ms_num += 1
        continue
      good_sgs.append(sgs_i)
      for j in top_words:
        if j[0] in string.punctuation:
          continue
        if j[0] in stopwords.words("english"):
          continue
        if j[0] in input_keywords:
          continue
        if j[0] in ["drug", "drugs"]:  # exclude these two for the drug dataset.
          continue
        if j[0][:2] == "##":  # the '##' by BERT indicates that is not a word.
          continue
        MLM_score[j[0]] += j[1]
    out = sorted(MLM_score, key = lambda x: MLM_score[x], reverse=True)
    out_tuple = [[x, MLM_score[x]] for x in out]
    if len(sgs) >=10:
      print("The percentage of uninformative masked sentences is {:d}/{:d} = {:.2f}%".format(
          skip_ms_num, len(sgs), float(skip_ms_num) / len(sgs) * 100
      ))
    return out, out_tuple, good_sgs
  def euphemism_detection(self, input_keywords, files, ms_limit, filter_uninformative):
    print("\n" + "*"*40 + " [Euphemism Detection] " + "*"*40)
    print("[util.py] Input Keyword: ", end="")
    print(input_keywords)
    print("[util.py] Extracting masked sentences for input keywords...")
    masked_sentence = []
    N = 0
    K = ms_limit
    MASK = " [MASK] "
    masked_sentence = self.data
    top_words, _, _ = self.MLM(
        masked_sentence,
        input_keywords,
        self.thres,
        filter_uninformative = filter_uninformative,
    )
    return top_words
  def run(self):
    input_keywords = [x.lower().strip() for x in self.given_keywords]

    top_words = self.euphemism_detection(input_keywords, self.data, ms_limit = 2000, filter_uninformative = 0)

    return top_words


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/seed_words.txt', 'r') as f:
  given_dogwhistles = f.readlines()

In [ ]:
given_dogwhistles = [x.strip().lower() for x in given_dogwhistles]

In [ ]:
euphemism_detector = DWdetector(given_dogwhistles, df["content"], 25600, "bert-large-uncased")

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
top_words = euphemism_detector.run()


**************************************** [Euphemism Detection] ****************************************
[util.py] Input Keyword: ['genderwoowoo', '#genderwoowoo', 'gender woo woo', '#milk', '#milk', '#milktwitter', '#womenwontwheesht', '#womenwontwheesht', "women won't wheesht", '109', '110', '1290', '1290 ad', '13%', '13/50', '13/52', '13/90', '14/88', '1488', '88', 'hh', '14 words', '23/16', '16/23', '2316', '1623', '41%', '41', '41 percent', '6mwe', "6 million wasn't enough", "six million wasn't enough", 'a leppo', 'abolish birthright citizenship', 'abolition of birthright citizenship', 'end birthright citizenship', 'absent fathers', 'absentee fathers', 'absent father', 'absentee father', 'lack of fathers', 'actual woman', 'actual women', 'adult human female', 'adult human females', 'afab trans woman', 'afab transgender woman', 'afab trans women', 'afab transgender women', 'affirmative action', 'agender', 'aiden', 'alarmism', 'climate alarmism', 'alarmist', 'alarmists', 'climate al

100%|██████████| 16258/16258 [05:39<00:00, 47.87it/s]

The percentage of uninformative masked sentences is 0/16258 = 0.00%


In [ ]:
top_words

[]

In [ ]:
drive.flush_and_unmount()